# A day in the life of a http connection
---

En este código, se pretende recrear todos los pasos que debe seguir un ordenador para poder realizar una conexión HTTP si no tuviera conocimiento de sus entornos. Los pasos son los siguientes:

1. Petición ARP broadcast para conocer la dirección de la interfaz del router
2. Petición DNS al servidor DNS para obtener la IP de la web a la que se quiere conectar
3. Conexión HTTP para solicitar el recurso

In [2]:
import os
os.environ['OPENAI_API_KEY'] = 'FILL'

import openai
from openai import OpenAI
client = OpenAI()

import random
import re
import pickle
import scapy
import toml
import ipaddress

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy
from scapy.layers.l2 import Ether, ARP
from scapy.layers.inet import IP
from scapy.all import DNS
from scapy.layers.http import *

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


#### Definitions

In [4]:
functioning ="Conversations"

Models_file = 'Models.jsonl'
pkl_saved = "../../data/ADayIn/pickle/ADayInTheLifeHTTP_Conversations_Fine_Tuned.pkl"
pcap_saved = "../../data/ADayIn/pcap/ADayInTheLifeHTTP_Conversations_generated_3_5_turbo-instruct_conv.pcap"

system_message = 'You are a new generation traffic generator. \
You are specilized in the {} protocol and {} generation using python and scapy. \
You are especially attentive to variables and different types of traffic.'

In [5]:
resource_pool = ["www.youtube.com", "www.marca.com", "www.uam.es", "www.bloomberg.com"]
IP_pool = ["93.184.216.34", "172.217.14.206", "151.101.1.69", "185.199.108.153", "104.244.42.65", "216.58.214.174", "34.235.250.23", "18.232.227.118", "23.239.22.188", "104.16.132.229"]
MAC_pool = [ "02:00:00:5a:3c:8f", "06:00:00:4b:7d:1e", "0a:00:00:6f:5a:2d", "0e:00:00:2c:9e:3a", "12:00:00:3e:1f:7b", "16:00:00:7b:8d:6c", "1a:00:00:1d:2a:4f", "1e:00:00:9e:3b:2d", "22:00:00:3a:7f:5e", "26:00:00:8b:4c:1f"]

In [24]:
def generateDay():
    #ALL
    IPori, IP_DnsServer, IP_HttpServer = random.choices(IP_pool, k=3)
    
    octets = IPori.split(".")
    octets[-1] = "1"
    IProuter = ".".join(octets)
    
    global MacOri
    MacOri= random.choices(MAC_pool, k=1)

    #DNS
    random_id = random.randint(0, 65535)
    random_resource = random.choices(resource_pool, k=1)

    #HTTP
    window1, window2 = random.randint(200, 6535), random.randint(200, 6535)
    port1 = random.randint(1025, 65535)
    port2 = 80
    code = 200
    ReasonPhrase = 'OK'

    ARP_conv_sum = "Source: MAC={}, IP={} // Destination: MAC={} // Wanted: IP= {}".format(
                        MacOri, IPori, "FF:FF:FF:FF:FF:FF", IProuter)

    DNS_conv_sum = "Source: IP= {} // Destination: IP= {} // Others: {:#06x} , resource= {} , response = {}".format(
                        IPori, IP_DnsServer, random_id, random_resource, IP_HttpServer)

    HTTP_conv_sum = "Source: IP={}, port={}, Window: {} // Destination: IP={}, port=80, Window: {} // Others: Host={}, Path="", Code={}, Reason_Phrase= {}".format(
                        IPori, port1, window1, IP_HttpServer, window2, random_resource, code, ReasonPhrase)
    
    
    return [ARP_conv_sum, DNS_conv_sum, HTTP_conv_sum]

In [26]:
def generate_Conversations(summaries):
    
    ctr= 1
    packets = []
    for summary in summaries:
        prompt = summary + "\n"
        protocol = obtain_Conversation_protocol(summary)

            
        if protocol == "ARP":
            messages=[{"role": "system", "content": system_message.format("ARP", functioning)},
            {"role": "user", "content": summary}]
            print("\t" + str(ctr)+". Using->  " + Models_list["Conversations"]["Specific"]["ARP"])
            completion = client.chat.completions.create(
            model=Models_list["Conversations"]["Specific"]["ARP"], 
            messages=messages,
            max_tokens=2600,
            temperature=0.1)

        elif protocol == "DNS":
            messages=[{"role": "system", "content": system_message.format("DNS", functioning)},
            {"role": "user", "content": summary}]
            print("\t" + str(ctr)+". Using->  " + Models_list["Conversations"]["Specific"]["DNS"])
            completion = client.chat.completions.create(
            model=Models_list["Conversations"]["Specific"]["DNS"], 
            messages=messages,
            max_tokens=2600,
            temperature=0.1)
        
        elif protocol == "HTTP":
            messages=[{"role": "system", "content": system_message.format("HTTP", functioning)},
            {"role": "user", "content": summary}]
            print("\t" + str(ctr)+". Using->  " + Models_list["Conversations"]["Specific"]["HTTP"])
            completion = client.chat.completions.create(
            model=Models_list["Conversations"]["Specific"]["HTTP"], 
            messages=messages,
            max_tokens=2600,
            temperature=0.1)
            
        
        responses.append(response(messages, completion))
        pickle.dump(responses, open(pkl_saved, "wb" ) )
        commands = responses[-1].completion.choices[0].message.content[1:]
        
        ldict = {}
        exec(commands,globals(),ldict)
        
        if protocol != "HTTP":
            if protocol == "ARP":
                MacRouter = ldict['pkt_2']["ARP"].hwsrc
                print(MacRouter)
                
                global eth_send
                eth_send = Ether(src=MacOri, dst=MacRouter)
                global eth_rec
                eth_rec = Ether(src=MacRouter, dst=MacOri)
                
                packets.append([ldict['pkt_1'], ldict['pkt_2']])
                del ldict
            elif protocol == "DNS":
                packets.append([eth_send/ldict['pkt_1'], eth_rec/ldict['pkt_2']])
                del ldict
           
        else:
            packets.append([eth_send/ldict['pkt_1'], eth_rec/ldict['pkt_2'], eth_send/ldict['pkt_3'], eth_send/ldict['pkt_4'], eth_rec/ldict['pkt_5'], eth_rec/ldict['pkt_6'], eth_send/ldict['pkt_7'], eth_send/ldict['pkt_8'], eth_rec/ldict['pkt_9'], eth_rec/ldict['pkt_10'], eth_send/ldict['pkt_11']])
            del ldict
        
        print("\t\t" + "Comand->  " + commands)
        
        ctr += 1
    return packets

In [28]:
def obtain_Conversation_protocol(summary):
    if "seq=" in summary:
        return "ICMP"
    elif "MAC=" in summary:
        return "ARP"
    elif "response" in summary:
        return "DNS"
    elif "Window:" in summary:
        return "HTTP"
    else:
        raise TypeError("Unsupported protocol.")

In [30]:
def write_pcap(packets, pcap_saved):
    for packet in packets:
        with open(pcap_saved, "ba+") as f:
            wrpcap(f, packet, append=True)
            
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

### Functioning

In [37]:
try:
    responses = pickle.load( open(pkl_saved, "rb" ))
except:
    responses = []

print("Number of previous responses: " + str(len(responses)))

Number of previous responses: 0


In [39]:
summaries = generateDay()

for summary in summaries:
    print(summary)

Source: MAC=['26:00:00:8b:4c:1f'], IP=34.235.250.23 // Destination: MAC=FF:FF:FF:FF:FF:FF // Wanted: IP= 34.235.250.1
Source: IP= 34.235.250.23 // Destination: IP= 104.244.42.65 // Others: 0xfbc2 , resource= ['www.uam.es'] , response = 185.199.108.153
Source: IP=34.235.250.23, port=45136, Window: 5199 // Destination: IP=185.199.108.153, port=80, Window: 6183 // Others: Host=['www.uam.es'], Path=, Code=200, Reason_Phrase= OK


In [41]:
with open(Models_file, 'r') as json_file:
    Models_list = json.load(json_file)

packets = generate_Conversations(summaries)
#print(packets)

print("\nGenerated {} packets.\nWriting them to {} ...".format(
        len(packets),
        pcap_saved))

write_pcap(packets, pcap_saved)
print("Done!")

	1. Using->  ft:gpt-3.5-turbo-1106:personal:convarpv2:9OUMMFtw
02:00:00:ad:f0:fa
		Comand->  pkt_1=scapy.Ether(src="26:00:00:8b:4c:1f", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="34.235.250.1", psrc="34.235.250.23", hwdst="00:00:00:00:00:00", hwsrc="26:00:00:8b:4c:1f")
time.sleep(random.randint(1,10)/1000000.0)
rand_MAC= "02:00:00:%02x:%02x:%02x" % (random.randint(0, 255),random.randint(0, 255),random.randint(0, 255))
pkt_2=scapy.Ether(src=rand_MAC, dst="26:00:00:8b:4c:1f")/scapy.ARP(op=2, psrc="34.235.250.1", hwsrc=rand_MAC, hwdst="26:00:00:8b:4c:1f", pdst="34.235.250.23")
	2. Using->  ft:gpt-3.5-turbo-1106:personal:convdnsv2:9OT57bdn
		Comand->  RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="34.235.250.23", dst="104.244.42.65")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0xfbc2, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.uam.es", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="104.244.42.65", dst="3